<a href="https://colab.research.google.com/github/JeLaKo/apple-tree-disease/blob/main/Milestone2_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading the data directly through Kaggle's API

In [1]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [2]:
!pip install -U -q kaggle==1.5.8

In [3]:
%cd /content/gdrive/MyDrive/MLP/

/content/gdrive/MyDrive/MLP


In [4]:
import os

In [5]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/MLP/"
! kaggle competitions download -c plant-pathology-2021-fgvc8

  7% 1.11G/14.9G [00:10<02:07, 116MB/s]
User cancelled operation


In [ ]:
! unzip -q plant-pathology-2021-fgvc8.zip -d train

Data processing


In [6]:
import pandas as pd
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

%cd /content/gdrive/MyDrive/MLP/train/

/content/gdrive/MyDrive/MLP/train


In [7]:
# import mlxtend package for confusion matrix
import mlxtend
                                                          
print(mlxtend.__version__) 

! pip install mlxtend --upgrade --no-deps

print(mlxtend.__version__) 

from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix


0.19.0
0.19.0


In [8]:
df = pd.read_csv('train.csv')
#print(df.head())

# df_test = pd.read_csv('sample_submission.csv')
# print(df_test.head())

In [9]:
new_labels = df['labels'].to_list()

for i in range(len(new_labels)):
  if new_labels[i] == 'scab frog_eye_leaf_spot complex' or new_labels[i] == 'scab frog_eye_leaf_spot':
    new_labels[i] = 'scab'
  elif new_labels[i] == 'frog_eye_leaf_spot complex':
    new_labels[i] = 'frog_eye_leaf_spot'
  elif new_labels[i] == 'powdery_mildew complex':
    new_labels[i] = 'powdery_mildew'
  elif new_labels[i] == 'rust complex' or new_labels[i] == 'rust frog_eye_leaf_spot':
    new_labels[i] = 'rust'


In [10]:
df['adjusted labels'] = np.array(new_labels)
df = df.drop('labels', axis = 1)

#print(df)

In [11]:
def sample(df, sample_size):
  """
  This function gets an equal sample inclusive of all classes from the input dataframe
  """
  df_sampled = []
  classes = df['adjusted labels'].unique()

  for i in classes:
      g = df[df['adjusted labels'] == i].sample(sample_size)
      df_sampled.append(g)

  df_sampled = pd.concat(df_sampled)
  return df_sampled

df_sampled = sample(df, 1184)
print(df_sampled)

                      image adjusted labels
10481  c89a4c8d729b945f.jpg         healthy
5463   a9bfd73e09b00cc1.jpg         healthy
16778  f56ad0b4ab965a82.jpg         healthy
17869  fc803b33128edee8.jpg         healthy
1383   8a966626bbb96c94.jpg         healthy
...                     ...             ...
7019   b23434724e9bd4cb.jpg  powdery_mildew
8481   bcd28d8f242f3730.jpg  powdery_mildew
3406   9b9832ecbd60702f.jpg  powdery_mildew
17746  fbc483ae483edc90.jpg  powdery_mildew
13636  e00f934d9e50ad5b.jpg  powdery_mildew

[7104 rows x 2 columns]


In [12]:
def one_hot(df):
  """
  This function returns all classes and combination of classes found in the input-dataframe, and returns
  the one-hot encoded version
  """
  one_hot = pd.get_dummies(df['adjusted labels'])
  df = df.drop('adjusted labels', axis = 1)
  df = df.join(one_hot)
  return df

df_onehot = one_hot(df_sampled)
#print(df_onehot)

# df_test = one_hot(df_test)
# print(df_test.head())

In [13]:
import matplotlib.pyplot as plt

from tensorflow.keras import layers, models, preprocessing

def train_and_evaluate(model, train_x, train_y, val_x, val_y, preprocess={}, epochs=20, augment={}):

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    train_gen = preprocessing.image.ImageDataGenerator(**preprocess, **augment)
    train_gen.fit(train_x) 

    val_gen = preprocessing.image.ImageDataGenerator(**preprocess)
    val_gen.fit(train_x)

    history = model.fit(train_gen.flow(train_x, train_y), epochs=epochs, 
                        validation_data=val_gen.flow(val_x, val_y))

    fig, axs = plt.subplots(1,2,figsize=(20,5)) 

    for i, metric in enumerate(['loss', 'accuracy']):
        axs[i].plot(history.history[metric])
        axs[i].plot(history.history['val_'+metric])
        axs[i].legend(['training', 'validation'], loc='best')

        axs[i].set_title('Model '+metric)
        axs[i].set_ylabel(metric)
        axs[i].set_xlabel('epoch')

    plt.show()

    print(f"Validation Accuracy: {model.evaluate(val_gen.flow(val_x, val_y))[1]}")

In [ ]:
# convert dictionary into a hashable type
df_dict = df_onehot.set_index('image').T.to_dict('list')

# gather names of all images in the image-directory
train_images = os.listdir('train_images/')

# # empty column for the actual images
# df_onehot["image_pixels"] = 0

images = []
# find corresponding key to image from directory, and add that image to a list
for image in df_dict.keys():
  if image in train_images:
    img = cv2.imread('train_images/' + image)
    img = cv2.resize(img, (96, 96)) 
    images.append(img)

labels = list(df_dict.values())

In [ ]:
images = np.array(images)
labels = np.array(labels)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(images, labels, test_size = 0.3, random_state=42)

In [ ]:

# Define Sequential model
model = models.Sequential()

# create convolutional layer and max pooling layer

model.add(layers.Conv2D(32, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.5), padding='same', input_shape=(96, 96, 3)))
model.add(layers.MaxPooling2D((2, 2)))

# create convolutional layer (larger) and max pooling layer
model.add(tf.keras.layers.Dropout(0.5))
model.add(layers.Conv2D(64, (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.5), padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization())

# flatten layers
model.add(layers.Flatten())
model.add(tf.keras.layers.Dropout(0.2))
model.add(layers.Dense(256, activation=tf.keras.layers.LeakyReLU(alpha=0.5)))

# apply softmax activation for final layer classification
model.add(tf.keras.layers.Dropout(0.2))
model.add(layers.Dense(6, activation='softmax'))

# normalize input data: set preprocesing dictionary
preprocess = {'featurewise_center': True, 'featurewise_std_normalization' : True}

# run training and evaluation function
train_and_evaluate(model, x_train, y_train, x_val, y_val, preprocess)

In [ ]:
model.summary()

In [ ]:
import seaborn as sns
import tensorflow as tf
from tensorflow import math as tfmath

classes = df['adjusted labels'].unique()
display(classes)
#x_test_flat = x_train.reshape(x_val.shape[0], (96, 96, 3)) # image_size

y_true = tf.argmax(y_val, axis=1)
y_pred = tf.argmax(model(x_val), axis=1)

conf_matrix = tfmath.confusion_matrix(y_true, y_pred, num_classes=6)

ax = sns.heatmap(conf_matrix, xticklabels=classes, yticklabels=classes)
ax.set(xlabel='Predicted Class', ylabel='Actual Class')
plt.show()

print(conf_matrix)

In [ ]:
! pip install mlxtend

In [ ]:
# define data for in confusion matrix
y_true = tf.argmax(y_val, axis=1)
y_pred = tf.argmax(model(x_val), axis=1)

# set labels for confusion matrix
labels =  df['adjusted labels'].unique()
display(labels)
# create and plot matrix
mtrx = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(conf_mat = mtrx, figsize=(8, 8), class_names=labels, colorbar=True, show_normed = True) # class_names=labels,